In [ ]:
# imports
import tensorflow as tf
import numpy as np
import os
from skimage.io import imread, imshow
from skimage.transform import resize
from skimage import color
from tensorflow.keras.models import load_model
from tensorflow.compat.v1 import get_default_graph
from tensorflow.compat.v1 import executing_eagerly_outside_functions
from tensorflow.compat.v1.losses import sparse_softmax_cross_entropy as sp_softmax_xent
from tensorflow.compat.v1.keras.optimizers import Adam
import matplotlib.pyplot as plt
import cv2

In [ ]:
# create X and y
IMG_HEIGHT = 224
IMG_WIDTH  = 224
CHANNELS = 3


training_images_names = os.listdir('data/membrane/train/image/')
training_masks_names = os.listdir('data/membrane/train/label/')

X = np.zeros((len(training_images_names),IMG_HEIGHT,IMG_WIDTH,CHANNELS),dtype='uint8')
y = np.zeros((len(training_masks_names),IMG_HEIGHT,IMG_WIDTH,1))

In [ ]:
# fill X and y
for i,n in enumerate(training_images_names):
    img = imread(f'data/membrane/train/image/{n}')
    img = resize(img,(IMG_HEIGHT,IMG_WIDTH,CHANNELS),mode='constant',preserve_range=True)
    fn = str(n.split('.')[0]) + '.png'
    mask = imread(f'data/membrane/train/label/{fn}')
    mask = resize(mask,(IMG_HEIGHT,IMG_WIDTH,1),mode='constant')
    
    X[i] = img
    y[i] = mask
    
X[0].shape

In [ ]:
# plot out an example
i = np.random.randint(0,len(X))
fig,(a1,a2)=plt.subplots(1,2)
a1.imshow(X[i])
a2.imshow(y[i].reshape(y[i].shape[:-1]))
print(y[i].reshape(y[i].shape[:-1]).shape)

In [ ]:
from model import unet
model=unet()

from keras.metrics import MeanIoU
iou_metric = MeanIoU(num_classes=2)

from dice import dice_loss
model.compile(optimizer = 'adam', loss=dice_loss, metrics=['accuracy', iou_metric])

In [ ]:
results = model.fit(X,y,batch_size=25,epochs=10)

In [ ]:
model.save('modelv3.h5')
